# 一、主要任务

想象一下，作为一个资深信贷风险专家，现针对中小企业贷款审批场景，需要主导构建标准化亲访评估体系。

具体实施路径如下：

首先，业务人员需对目标企业开展实地亲访，重点采集生产经营全貌数据。我们将以首批手工记录的亲访笔记为原始素材，调用ERNIE-4.5-300B-A47B大语言模型进行结构化重构——从零散的手工记录中提炼关键风险维度，通过多轮迭代优化形成覆盖生产流程、财务健康度、供应链稳定性、市场竞争力等核心模块的完备亲访要求体系。

该迭代过程将遵循"数据采集-特征提取-规则沉淀-表单映射"的闭环逻辑：模型首先解析原始记录中的非结构化信息，识别出设备状态、库存周转、用工情况等关键观测点；随后通过风险特征工程构建量化评估维度，如产能利用率波动阈值、应收账款、账期预警等。最终基于这些维度生成具备操作指引性的亲访表单。

最终输出的亲访表单将实现三大核心功能：一是标准化采集模板，包含必填项、选填项及异常值标注规则；二是风险评估矩阵，将现场观察指标与信用评级维度建立映射关系；三是动态校验机制，通过逻辑校验规则确保数据采集的完整性。表单将采用模块化设计，支持调整不同的观测重点，在亲访现场实时反馈数据采集质量。

通过这一体系化建设，我们将实现从经验驱动到数据智能驱动的亲访评估升级，既保障风险评估的颗粒度与穿透力，又提升业务人员的现场作业效率，最终构建起可复制、可验证的中小企业信贷风控基础设施。


# 二、快速开始

## 2.1 模型介绍

后续我们要求模型需要强大的文本理解和生成能力，这里选用模型**ERNIE-4.5-300B-A47B**：

[https://aistudio.baidu.com/modelsdetail/30641/intro](https://aistudio.baidu.com/modelsdetail/30641/intro)

* 模型名称

ERNIE-4.5-300B-A47B

* 参数规模

300B

* 类型

Text Generation

* 应用场景

主力旗舰版，研究 & 工业应用


## 2.2 环境初始

In [1]:

# 通过.env配置
import os
from dotenv import load_dotenv
load_dotenv()
API_KEY = os.getenv("API_KEY")

# pip install openai
import json
from openai import OpenAI


## 2.3 系统提示词

In [2]:


# 定义系统提示词，用于指导AI完成任务
_KEY1 = "亲访要求"
_KEY2 = "亲访记录"

_SYSTEM = f"""
你是一个信贷风险专家，现在将为不同的中小企业审批下发贷款申请。
业务人员会去到对应企业的实际生产经营场地亲身访问，以下简称亲访。
并按照{_KEY1}，会进行一个{_KEY2}。
需要不断地归纳和总结，并精简，得到亲访过程切实关注的重点，返回到json的亲访要求字段中。

输入：
{{'{_KEY1}': {{}}, '{_KEY2}': ''}}
输出：
{{'{_KEY1}': {{}}}}
用json的模式返回。
"""
print(_SYSTEM)


# 收集亲访要求，并录入到criteriaR中
criteriaR = {
    0: {}
}




你是一个信贷风险专家，现在将为不同的中小企业审批下发贷款申请。
业务人员会去到对应企业的实际生产经营场地亲身访问，以下简称亲访。
并按照亲访要求，会进行一个亲访记录。
需要不断地归纳和总结，并精简，得到亲访过程切实关注的重点，返回到json的亲访要求字段中。

输入：
{'亲访要求': {}, '亲访记录': ''}
输出：
{'亲访要求': {}}
用json的模式返回。



## 2.4 模型配置

使用以下配置：

```python
stream=False,
extra_body={
    "penalty_score": 1
},
max_completion_tokens=12000,
temperature=0.8,
top_p=0.8,
frequency_penalty=0,
presence_penalty=0
```


In [3]:


def askAI(IN_json, IN_new):
    client = OpenAI(
        api_key=API_KEY, 
        base_url="https://aistudio.baidu.com/llm/lmapi/v3",
    )

    _i = {
        _KEY1: IN_json,
        _KEY2: IN_new,
    }
    
    response = client.chat.completions.create(
        model="ernie-4.5-turbo-128k-preview",
        messages=[
            {   
                "role": "system", 
                "content": _SYSTEM,
            },
            {
                "role": "user",
                "content": json.dumps(_i, ensure_ascii=False),
            }
        ],

        stream=False,
        extra_body={
            "penalty_score": 1
        },
        max_completion_tokens=12000,
        temperature=0.8,
        top_p=0.8,
        frequency_penalty=0,
        presence_penalty=0
    )

    try:
        _j = json.loads(
            response.choices[0].message.content.replace("`","").replace("json","")
        )
        return {_KEY1: _j[_KEY1]}
    except Exception as e:
        print("ERROR", e)
        _j = criteriaR[0]
    
    return _j


def printJSON(J): 
    # 打印美化json格式
    print(json.dumps(J, sort_keys=True, indent=2, ensure_ascii=False))


def run(_new):
    # 每次运行一个案例，使用上一条criteria，并更新一次criteriaR字典
    _out = askAI(IN_json=criteriaR[_new["ID"]-1], IN_new=_new["RECORD"])
    print(f'> {_new["ID"]}/{len(criteriaR)}')
    printJSON(_out)
    criteriaR[_new["ID"]] = _out



## 2.5 运行迭代

**！！！注意！！！**

*以下案例，我们参考了实际案例，但经过资深专家深度挖掘，进行了相关信息脱敏、概念转换和案例融合。*

*如有雷同，纯属巧合。*

**！！！注意！！！**


### 2.5.1 从0开始

**这里定义ID为1，初始化状态下，我们传入的亲访要求criteriaR[0]为空的{}。**

In [4]:
%%time
run({"ID": 1,
"RECORD": """
对于xxx，建议予以拒绝，相关申请已提交。具体理由如下：
1. 其亲访地址与营业执照上的地址不一致，且无法提供租赁合同及交租凭证来佐证其真实经营情况；
2. 无法提供经营单据以证明yyy公司有实际经营行为，同时现场既无公司招牌，也无员工在场，无法佐证其实际经营状况。
"""})

> 1/1
{
  "亲访要求": {
    "地址验证": [
      "核实亲访地址是否与营业执照上的地址一致",
      "如不一致，需提供租赁合同及交租凭证以佐证真实经营情况"
    ],
    "经营证明": [
      "要求提供经营单据以证明公司有实际经营行为",
      "现场应存在公司招牌及员工，以佐证实际经营状况"
    ]
  }
}
CPU times: user 201 ms, sys: 52.4 ms, total: 253 ms
Wall time: 3.68 s


### 2.5.2 持续优化

**这里定义ID为2，我们传入的亲访要求criteriaR[1]，即上一手刚刚生成的亲访要求。**


In [5]:
%%time
run({"ID": 2,
"RECORD": """
关于xxx的申请，综合评估后建议予以通过，相关申请材料已提交。同时，建议重点关注以下第1、3、5项内容：
申请人基本情况：专科学历，持有A城市户口，未婚状态，名下无房产及车辆，在品牌策划领域拥有十年从业经验，其名下公司已稳定经营五年。
具体审核要点如下：
1. 营业执照注册地址与实际经营场所吻合，该团队属于留学生创业扶持项目入孵团队，已提供有效租赁合同。目前办公现场有2名工作人员，据称另有分支办公场所，公司总人数达14人。
2. 企业持续开展实质性经营活动，提供了2016-2025年度的完整经营单据及项目合同，并现场演示了转型后开发的小程序产品。
3. 企业负责人坦陈受疫情影响及公司战略转型因素，纳税额由12万元降至6万元。
4. 借款人曾在中国农业银行办理98万元贷款业务，当前剩余未偿金额为50万元。
5. 借款人自评资质优良，对其他金融机构拒贷表示困惑，现申请50万元贷款，专项用于转型后技术运维团队的日常资金周转。
"""})

> 2/2
{
  "亲访要求": {
    "地址验证": [
      "核实亲访地址是否与营业执照上的地址一致",
      "如不一致，需提供租赁合同及交租凭证以佐证真实经营情况",
      "现场查看并确认是否存在分支办公场所及人员分布情况"
    ],
    "经营证明": [
      "要求提供经营单据以证明公司有实际经营行为",
      "现场应存在公司招牌及员工，以佐证实际经营状况",
      "查看并确认公司提供的年度经营单据及项目合同，以验证持续经营情况"
    ]
  }
}
CPU times: user 6.02 ms, sys: 11.7 ms, total: 17.7 ms
Wall time: 4 s


In [6]:
%%time
run({"ID": 3,
"RECORD": """
关于xxx的申请，经综合评估后建议予以通过，相关申请材料已正式提交。
申请人基本情况：中专学历，已婚状态，名下拥有车辆，在相关领域具备23年从业经验，经营地址与注册地址完全一致。
现场核验情况：经营场所内有员工正在进行库存管理及发货作业，其配偶亦参与日常经营管理，确认存在实质性经营活动。
证明材料情况：已提供有效租赁合同及完整的经营单据作为佐证。
资金用途说明：拟申请53万元贷款，专项用于库存采购及运营规模扩张。
"""})

> 3/3
{
  "亲访要求": {
    "地址验证": [
      "核实亲访地址与营业执照地址是否一致，不一致需提供租赁合同及交租凭证",
      "确认有无分支办公场所及人员分布"
    ],
    "经营证明": [
      "现场查看是否存在公司招牌及员工，佐证实际经营",
      "查看并确认公司提供的经营单据及项目合同，验证持续经营"
    ]
  }
}
CPU times: user 20.5 ms, sys: 65 μs, total: 20.6 ms
Wall time: 3.7 s


In [7]:
%%time
run({"ID": 4,
"RECORD": """
关于xxx的申请，经综合评估建议予以通过，需重点关注第1项情况，相关材料已提交。
申请人背景：高中学历，由电工转型从事装饰硬装行业，公司已持续经营8年。
重点核查事项：
1. 现场核查发现无固定员工，申请人解释主要雇佣临时散工，现场仅存零星库存，企业标识（logo）显示不显著。
其他核查情况：
2. 已提供完整租赁合同、租金支付凭证及经营业务单据。
3. 申请人自述经营策略偏向保守，客户群体以熟人介绍为主。
4. 贷款态度谨慎，现申请33万元贷款，专项用于装饰工程项目垫资。
"""})

> 4/4
{
  "亲访要求": {
    "地址验证": [
      "核实亲访地址与营业执照地址一致性，不一致需提供租赁合同及交租凭证",
      "确认有无分支办公场所及人员分布"
    ],
    "经营稳定性": [
      "关注企业用工模式（如临时散工）对经营稳定性的影响",
      "评估企业标识显著性对业务开展的影响"
    ],
    "经营证明": [
      "查看公司招牌及员工在岗情况，确认实际经营状态",
      "核查经营单据及项目合同，验证持续经营能力"
    ]
  }
}
CPU times: user 17 ms, sys: 2.77 ms, total: 19.8 ms
Wall time: 5.34 s


In [8]:
%%time
run({"ID": 5,
"RECORD": """
关于xxx的申请，经综合评估建议予以通过，相关申请材料已提交。具体评估要点如下：
1. 学历资质：申请人具备本科学历，已提供学历证书，且学信网可查证。
2. 经营状况：申请人持续开展正常经营活动，已提供有效租赁合同作为经营场所证明。
3. 家庭情况：婚姻状况稳定，育有一名18个月大的子女。需注意本次贷款申请未告知其配偶。
4. 资产情况：在A城市B区拥有房产（登记在其配偶名下）。
5. 申请用途：贷款资金专项用于经营性资金周转。申请人名下拥有4台运输货车，因快递行业月结回款周期较长，存在阶段性资金需求。
"""})

> 5/5
{
  "亲访要求": {
    "其他关注点": [
      "核实贷款申请用途与实际经营需求是否匹配",
      "关注申请人家庭情况对贷款偿还能力的影响（如婚姻状况稳定性、家庭资产情况等）"
    ],
    "地址验证": [
      "核实亲访地址与营业执照地址一致性，不一致需提供租赁合同及交租凭证",
      "确认有无分支办公场所及人员分布"
    ],
    "经营稳定性": [
      "关注企业用工模式（如临时散工）对经营稳定性的影响",
      "评估企业标识显著性对业务开展的影响"
    ],
    "经营证明": [
      "查看公司招牌及员工在岗情况，确认实际经营状态",
      "核查经营单据及项目合同，验证持续经营能力"
    ]
  }
}
CPU times: user 17.2 ms, sys: 1.02 ms, total: 18.2 ms
Wall time: 6.23 s


In [9]:
%%time
run({"ID": 6,
"RECORD": """
关于xxx的申请，经综合评估建议予以通过，相关材料已提交，需重点关注第3、4项情况。具体评估要点如下：
1. 从业背景：初中学历，离异状态，名下拥有房产及车辆。自2018年起从事xx屏等电子类产品经营，公司已持续运营10年。
2. 经营场所：注册地址与实际经营地址一致，兼具办公及仓储功能，当前库存价值约450万元。另在B区设有经营场地，因相关经营单据留存于该处未予提供。
3. 财务证明：已提供租赁合同、租金支付凭证及银行流水，其中安平银行账户显示较大额还款记录（需进一步核查资金用途）。
4. 证件核验：房产证、车辆行驶证等权属证明仅提供照片件（建议补充原件核验）。
5. 申请用途：申请人认为xx屏市场价格呈上升趋势，拟申请30万元贷款用于持续囤积库存。
"""})

> 6/6
{
  "亲访要求": {
    "其他关注点": [
      "核实贷款申请用途与实际经营需求是否匹配，关注市场趋势对贷款用途的影响",
      "关注申请人家庭情况对贷款偿还能力的影响（如婚姻状况稳定性、家庭资产情况等）",
      "核查银行流水较大额还款记录的资金用途"
    ],
    "地址验证": [
      "核实亲访地址与营业执照地址一致性，不一致需提供租赁合同及交租凭证",
      "确认有无分支办公场所及人员分布，需查看分支场所相关证明"
    ],
    "经营稳定性": [
      "关注企业用工模式（如临时散工）对经营稳定性的影响",
      "评估企业标识显著性对业务开展的影响"
    ],
    "经营证明": [
      "查看公司招牌及员工在岗情况，确认实际经营状态",
      "核查经营单据及项目合同，验证持续经营能力，若单据留存分支场所需特别说明"
    ],
    "财务与权属证明": [
      "核查租赁合同、租金支付凭证及银行流水等财务证明",
      "房产证、车辆行驶证等权属证明需提供原件核验"
    ]
  }
}
CPU times: user 13 ms, sys: 4.25 ms, total: 17.3 ms
Wall time: 9.49 s


### 2.5.3 拒绝案例

**这里传入比较复杂的拒绝案例，充分向LLM传递亲访过程中需要避雷的点。**


In [10]:
%%time
run({"ID": 7,
"RECORD": """
关于xxx的申请，经综合评估建议予以拒绝，具体评估情况如下：
申请人背景：本科学历，未婚状态，拥有16年品牌相关行业从业经验。申请人自述因个人业务规模扩大，出于信用及合规考虑决定注册公司。
经营异常情况：
1. 地址存疑：营业执照注册地址与实际经营场所不一致，申请人解释为近期搬迁，但仅提供2021年3月起租的租赁合同。
2. 人员配置异常：宣称公司有7名员工，但现场仅见财务及设计师2人，办公电脑散乱且未完成安装调试。
3. 品牌标识混乱：公司主要标识为同楼层其他企业（曾合作方）的logo，仅存在临时性小logo。
4. 经营证据薄弱：
    提供2025年3月经营单据，但单据中包含其他公司业务记录
    自称拥有大客户支撑27万元业绩，但无法提供有效合同佐证
5. 项目真实性存疑：拟申请贷款用于与省教育厅合作的新项目人力扩充，但无法核实项目真实性及公司实际经营状况。
风险总结：申请人存在经营地址不符、人员配置与宣称不符、品牌标识混淆、经营证据交叉等重大疑点，且新项目合作真实性无法验证，整体经营状况存疑。
"""})

> 7/7
{
  "亲访要求": {
    "其他关注点": [
      "核实贷款申请用途与实际经营需求匹配度，评估新项目合作真实性及市场前景",
      "关注申请人家庭稳定性（如婚姻状况）及家庭资产状况对贷款偿还能力的影响",
      "核查银行流水异常记录（如大额还款）的资金用途合理性"
    ],
    "地址验证": [
      "核实亲访地址与营业执照地址一致性，不一致需提供最新租赁合同及交租凭证",
      "确认分支办公场所存在性及人员分布，查看分支场所相关证明"
    ],
    "经营稳定性": [
      "评估企业用工模式合理性，关注临时用工对经营稳定性的潜在影响",
      "核查企业标识显著性及独立性，避免品牌混淆风险"
    ],
    "经营证明": [
      "查看公司招牌、员工在岗情况及办公环境，确认实际经营状态",
      "核查近期经营单据、项目合同，确保与主营业务相关且无交叉混淆"
    ],
    "财务与权属证明": [
      "核查租赁合同、租金支付凭证、银行流水等财务证明时效性及一致性",
      "房产证、车辆行驶证等权属证明需提供原件核验，确认权属清晰"
    ]
  }
}
CPU times: user 27.7 ms, sys: 0 ns, total: 27.7 ms
Wall time: 9.93 s


In [11]:
%%time
run({"ID": 8,
"RECORD": """
关于xxx的申请，经综合评估建议予以通过，相关材料已提交。具体评估要点如下：
申请人背景：大专学历，已婚育有子女，名下拥有车辆，与配偶共同经营企业。
经营情况：
1. 行业经验：具备15年精密模具加工行业经验，2009年自主创立公司，现与同层的yyy公司分租场地并开展业务合作。
2. 资金周转：采用甲方预付30%原材料款的结算模式，无需自行垫资，且因定制化生产要求无库存积压。
3. 证明材料：
    已提供分租场地的租赁合同及付款凭证
    补充提交300万元银行流水（主要采用对公账户结算）
4. 贷款用途：拟申请贷款用于引进日本进口生产设备，申请人说明现有设备产能已饱和，现场核查显示员工在岗、设备运转正常，确认处于实质性生产状态。
风险控制：经营模式清晰，资金流水可追溯，贷款用途与实际生产需求匹配，夫妻共同经营有助于增强还款保障。
"""})

> 8/8
{
  "亲访要求": {
    "其他关注点": [
      "核实贷款申请用途与实际经营需求匹配度，评估新项目合作真实性及市场前景",
      "关注申请人家庭稳定性（如婚姻状况）及家庭资产状况对贷款偿还能力的影响",
      "核查银行流水异常记录（如大额还款）的资金用途合理性，确保资金流动合规"
    ],
    "地址验证": [
      "核实亲访地址与营业执照地址一致性，不一致需提供最新租赁合同及交租凭证",
      "确认分支办公场所存在性及人员分布，查看分支场所相关证明（如有）"
    ],
    "经营稳定性": [
      "评估企业用工模式合理性，关注临时用工对经营稳定性的潜在影响",
      "核查企业标识显著性及独立性，避免品牌混淆风险"
    ],
    "经营证明": [
      "查看公司招牌、员工在岗情况及办公环境，确认实际经营状态",
      "核查近期经营单据、项目合同，确保与主营业务相关且无交叉混淆"
    ],
    "财务与权属证明": [
      "核查租赁合同、租金支付凭证、银行流水等财务证明时效性及一致性",
      "房产证、车辆行驶证等权属证明需提供原件核验，确认权属清晰",
      "银行流水需重点核查大额资金流动的合理性"
    ]
  }
}
CPU times: user 17.2 ms, sys: 84 μs, total: 17.3 ms
Wall time: 10.1 s


### 2.5.4 复杂案例

**这里传入比较复杂的亲访案例，传递更多亲访细节。**


In [12]:
%%time
run({"ID": 9,
"RECORD": """
关于xxx的申请，经综合评估建议予以拒绝，相关材料已提交。具体风险点如下：
申请人背景：大专学历，已婚育有子女，名下拥有车辆。
核心疑点：
1. 经营场所异常：营业执照注册地址为自住房，申请人解释已搬迁，但现场核查发现：
   - 无实际产品陈列
   - 无员工在岗
   - 宣称"基本在外跑客户"但缺乏出差记录佐证
2. 合作模式存疑：
   - 声称与yyy公司（朋友企业）分租场地并开展合作
   - 业务模式为代生产，但现场无生产设备及库存
   - 品牌标识混乱，存在借用合作方logo情况
3. 经营证据缺失：
   - 宣称"无纸化办公"却无法提供完整经营单据
   - 仅提供单方面电脑图片数据，无第三方凭证
   - 微信流水金额较小，且拒绝提供银行流水
4. 贷款用途模糊：
   - 配偶对贷款事宜回复含糊，家庭共同决策存疑
   - 拟申请贷款用于材料垫付，但未来需求预测缺乏数据支撑
风险总结：申请人存在经营地址与注册地址不符、合作模式不清晰、经营证据缺失、贷款用途不明确等重大风险点，整体经营真实性无法核实，建议不予通过。
"""})

> 9/9
{
  "亲访要求": {
    "其他关注点": [
      "核实贷款申请用途与实际经营需求匹配度，评估新项目合作真实性及市场前景",
      "关注申请人家庭稳定性（如婚姻状况）及家庭资产状况对贷款偿还能力的影响",
      "核查银行流水异常记录（如大额还款）的资金用途合理性，确保资金流动合规",
      "重点核查经营证据完整性，包括但不限于生产设备、库存、完整经营单据等"
    ],
    "地址验证": [
      "核实亲访地址与营业执照地址一致性，不一致需提供最新租赁合同及交租凭证",
      "确认分支办公场所存在性及人员分布，查看分支场所相关证明（如有）",
      "重点核查经营场所实际产品陈列、员工在岗情况，避免以\"外出跑客户\"为由规避检查"
    ],
    "经营稳定性": [
      "评估企业用工模式合理性，关注临时用工对经营稳定性的潜在影响",
      "核查企业标识显著性及独立性，避免品牌混淆风险",
      "重点核查合作模式真实性，确认是否存在场地分租、品牌借用等风险点"
    ],
    "经营证明": [
      "查看公司招牌、员工在岗情况及办公环境，确认实际经营状态",
      "核查近期经营单据、项目合同，确保与主营业务相关且无交叉混淆",
      "要求提供完整经营单据及第三方凭证，拒绝单方面电脑图片数据"
    ],
    "财务与权属证明": [
      "核查租赁合同、租金支付凭证、银行流水等财务证明时效性及一致性",
      "房产证、车辆行驶证等权属证明需提供原件核验，确认权属清晰",
      "银行流水需重点核查大额资金流动的合理性，拒绝提供时视为重大风险"
    ]
  }
}
CPU times: user 16.1 ms, sys: 3.21 ms, total: 19.3 ms
Wall time: 13.9 s


### 2.5.5 口头案例

**这里传入没有太规范化，甚至充满口头话术的案例，这能更加充分体现LLM的文本理解能力。**


In [13]:
%%time
run({"ID": 10,
"RECORD": """
xxx
建议予以拒绝，相关申请已提交，具体情况如下：
1、涉及农产品转销业务，对于婚姻状况的回复含混不清。
2、地址信息无误，其自称较早进驻A城市某科技园内，虽提供了租赁证明，然而现场既无员工也无库存，办公室内仅有一台较新的电脑，桌椅却布满灰尘。
3、该主体表示去年销售额约合40W左右，自称经营状况不佳，基本没有盈利，微信流水金额较小，未来打算转到抖音等平台进行销售。
4、称有一笔26W先息后本的贷款尚未结清，现拟申请20W贷款，但未明确说明贷款的具体用途。
"""})

> 10/10
{
  "亲访要求": {
    "其他关注点": [
      "核实贷款申请用途与实际经营需求匹配度，评估新项目合作真实性及市场前景",
      "关注申请人婚姻状况及家庭资产状况对贷款偿还能力的影响",
      "核查银行流水异常记录（如大额还款）的资金用途合理性",
      "重点核查经营证据完整性，包括生产设备、库存、完整经营单据等"
    ],
    "地址验证": [
      "核实亲访地址与营业执照地址是否一致，不一致需提供最新租赁合同及交租凭证",
      "确认经营场所实际存在，核查员工在岗及产品陈列情况，避免空壳公司"
    ],
    "经营稳定性": [
      "评估企业用工模式，关注临时用工对经营稳定性的影响",
      "核查企业标识显著性及独立性，避免品牌混淆",
      "确认合作模式真实性，核查是否存在场地分租、品牌借用等风险"
    ],
    "经营证明": [
      "查看公司招牌、办公环境及员工在岗情况，确认实际经营状态",
      "核查近期经营单据、项目合同，确保与主营业务相关且真实有效",
      "要求提供完整经营单据及第三方凭证，拒绝仅提供电脑图片数据"
    ],
    "财务与权属证明": [
      "核查租赁合同、租金支付凭证、银行流水等财务证明的时效性及一致性",
      "房产证、车辆行驶证等权属证明需提供原件核验，确认权属清晰无争议",
      "重点核查银行流水大额资金流动的合理性，拒绝提供时视为重大风险信号"
    ]
  }
}
CPU times: user 9.95 ms, sys: 7.66 ms, total: 17.6 ms
Wall time: 11.9 s


In [14]:
%%time
run({"ID": 11,
"RECORD": """
xxx 建议拒绝，相关申请已提交，具体情况如下：
该人员为初中学历，已婚且有子女。
1、地址信息一致，其声称有6名员工，从事模具生产业务。实地查看发现，一层作为车间，既无员工也无生产活动，仅有3台设备（其中1台很少使用），阁楼作为办公室。
2、其表示“z”相关的业务占销售额的70%，并称主要由配偶负责沟通，自己不太了解模具五金等生产细节。
3、称配偶外出，现场仅有2人，另一名男子（自称是同事）见到实地访问人员时反应“又是贷款”，但借款人否认有其他贷款申请。
4、拟申请30W贷款，但对于配偶是否知晓贷款事宜以及资金具体用途，表述均较为模糊。
"""})

> 11/11
{
  "亲访要求": {
    "其他关注点": [
      "核实贷款申请用途与实际经营需求匹配度，评估新项目合作真实性及市场前景",
      "关注申请人婚姻状况及家庭资产状况对贷款偿还能力的影响",
      "核查银行流水异常记录（如大额还款）的资金用途合理性",
      "确认配偶对贷款事宜的知情度及资金用途的明确性",
      "关注现场人员对贷款访问的反应，排查多头贷款风险"
    ],
    "地址验证": [
      "核实亲访地址与营业执照地址是否一致，不一致需提供最新租赁合同及交租凭证",
      "确认经营场所实际存在，核查员工在岗及产品陈列情况，避免空壳公司"
    ],
    "经营稳定性": [
      "评估企业用工模式，关注临时用工对经营稳定性的影响",
      "核查企业标识显著性及独立性，避免品牌混淆",
      "确认合作模式真实性，核查是否存在场地分租、品牌借用等风险",
      "核实主营业务占比及负责人对业务细节的了解程度"
    ],
    "经营证明": [
      "查看公司招牌、办公环境及员工在岗情况，确认实际经营状态",
      "核查近期经营单据、项目合同，确保与主营业务相关且真实有效",
      "要求提供完整经营单据及第三方凭证，拒绝仅提供电脑图片数据",
      "实地核查生产设备、库存等经营证据完整性"
    ],
    "财务与权属证明": [
      "核查租赁合同、租金支付凭证、银行流水等财务证明的时效性及一致性",
      "房产证、车辆行驶证等权属证明需提供原件核验，确认权属清晰无争议",
      "重点核查银行流水大额资金流动的合理性，拒绝提供时视为重大风险信号"
    ]
  }
}
CPU times: user 28.2 ms, sys: 3.2 ms, total: 31.4 ms
Wall time: 12.6 s


In [15]:
%%time
run({"ID": 12,
"RECORD": """
xxx建议通过，具体评估情况如下：
1. 申请人处于真实经营状态。受去年疫情影响，曾暂停营业6个月，租赁场地仅减免了2个月租金，当前资金压力较大。
2. 拟申请50万资金，用于企业资金周转。
3. 申请人婚姻状况稳定良好，育有2个小孩，拥有A城市B区户口，并持有小产权房。
4. 申请人太太与合伙人共同经营其他校区，开展小区内幼儿早教业务，平均每月服务30个小孩，收费标准为5230元/人/月。
5. 申请人关联的yyy公司依托于xx教育集团，作为教育品牌需在教育局备案。该公司主要提供艺术类及学科类课程，平均收费为82元/节课。
6. 申请人目前有2笔贷款，分别为车贷约22万，以及另一家银行的66万贷款。
建议综合考量其负债比例，给予适当贷款额度。
"""})

> 12/12
{
  "亲访要求": {
    "其他关注点": [
      "核实贷款申请用途与实际经营需求匹配度",
      "关注申请人婚姻状况及家庭资产状况对贷款偿还能力的影响",
      "核查银行流水异常记录（如大额还款）的资金用途合理性",
      "关注现场人员对贷款访问的反应，排查多头贷款风险"
    ],
    "地址验证": [
      "核实亲访地址与营业执照地址是否一致，不一致需提供最新租赁合同及交租凭证",
      "确认经营场所实际存在，核查员工在岗及产品陈列情况"
    ],
    "经营稳定性": [
      "评估企业用工模式，关注临时用工对经营稳定性的影响",
      "确认主营业务占比及负责人对业务细节的了解程度"
    ],
    "经营证明": [
      "查看公司招牌、办公环境及员工在岗情况，确认实际经营状态",
      "核查近期经营单据、项目合同，确保与主营业务相关且真实有效",
      "实地核查生产设备、库存等经营证据完整性"
    ],
    "财务与权属证明": [
      "核查租赁合同、租金支付凭证、银行流水等财务证明的时效性及一致性",
      "房产证、车辆行驶证等权属证明需提供原件核验，确认权属清晰无争议",
      "重点核查银行流水大额资金流动的合理性"
    ]
  }
}
CPU times: user 16.5 ms, sys: 876 μs, total: 17.4 ms
Wall time: 7.36 s


In [16]:
%%time
run({"ID": 13,
"RECORD": """
xxx建议拒绝，具体缘由如下：
1. 申请人自称研究生学历，目前处于离异状态。
2. 无法核实其是否真实开展经营活动。
3. 其公司位于A城市B区的一条小巷子里的私人楼房内，周边环境不佳。
4. 申请人对于自身经营内容无法清晰阐述。
建议进一步了解该进件的来源，避免此类进件再次出现。
"""})

> 13/13
{
  "亲访要求": {
    "其他关注点": [
      "核实贷款申请用途与实际经营需求匹配度",
      "关注申请人婚姻状况及家庭资产状况对贷款偿还能力的影响",
      "核查银行流水异常记录（如大额还款）的资金用途合理性",
      "关注现场人员对贷款访问的反应，排查多头贷款风险",
      "评估申请人对自身经营内容的熟悉程度"
    ],
    "地址验证": [
      "核实亲访地址与营业执照地址是否一致，不一致需提供最新租赁合同及交租凭证",
      "确认经营场所实际存在且环境符合经营需求",
      "核查经营场所内员工在岗及产品陈列情况"
    ],
    "经营稳定性": [
      "评估企业用工模式，关注临时用工对经营稳定性的影响",
      "确认主营业务占比，评估负责人对业务细节的了解程度"
    ],
    "经营证明": [
      "查看公司招牌、办公环境及员工在岗情况，确认实际经营状态",
      "核查近期经营单据、项目合同，确保与主营业务相关且真实有效",
      "实地核查生产设备、库存等经营证据完整性"
    ],
    "财务与权属证明": [
      "核查租赁合同、租金支付凭证、银行流水等财务证明的时效性及一致性",
      "房产证、车辆行驶证等权属证明需提供原件核验，确认权属清晰无争议",
      "重点核查银行流水大额资金流动的合理性"
    ]
  }
}
CPU times: user 14.8 ms, sys: 4.53 ms, total: 19.4 ms
Wall time: 9.81 s


In [17]:
%%time
run({"ID": 14,
"RECORD": """
xxx建议通过，具体评估情况如下：
1. 申请人处于真实经营状态，拟申请32万资金用于企业资金周转，且目前无其他贷款。
2. 申请人主要经营装修行业，同时涉及原材料设备的研发，具体业务包括吊顶、瓷片砖等，每项工程均能提供正常发票。
3. 公司租赁合同方面，目前仅有旧合同，有效期至2026年2月，新合同尚未签订。
4. 申请人婚姻状况稳定正常，太太知悉并同意该笔贷款申请，育有3个小孩。
5. 申请人非A城市户口，在B城市拥有房产。
"""})

> 14/14
{
  "亲访要求": {
    "其他关注点": [
      "核实贷款申请用途与实际经营需求匹配度",
      "关注申请人婚姻状况及家庭资产状况对贷款偿还能力的影响",
      "关注现场人员对贷款访问的反应，排查多头贷款风险"
    ],
    "地址验证": [
      "核实亲访地址与营业执照地址一致性，不一致需提供最新租赁合同及交租凭证",
      "确认经营场所实际存在且环境符合经营需求"
    ],
    "经营稳定性": [
      "确认主营业务占比，评估负责人对业务细节的了解程度"
    ],
    "经营证明": [
      "查看公司招牌、办公环境及员工在岗情况，确认实际经营状态",
      "核查近期经营单据、项目合同，确保与主营业务相关且真实有效"
    ],
    "财务与权属证明": [
      "核查租赁合同、租金支付凭证等财务证明的时效性及一致性",
      "重点核查银行流水大额资金流动的合理性"
    ]
  }
}
CPU times: user 14.1 ms, sys: 4.44 ms, total: 18.5 ms
Wall time: 7.67 s


In [18]:
%%time
run({"ID": 15,
"RECORD": """
xxx建议通过，具体评估情况如下：
1. 申请人处于真实经营状态，是网店平台的第三方供应商，供货范围涵盖欧洲、日本、澳洲等地，主要经营女性头饰和户外体育用品。
2. 申请人公司现有员工5人（其中包含1名合伙人）。公司注册地址与实际经营地址不一致，主要原因是原注册地址被用作仓库。
3. 申请人婚姻状况稳定正常，太太在B城市，育有子女。申请人非A城市户口，在A城市无房产，在B城市拥有房产，在A城市拥有小车。
4. 公司招牌因外墙不允许打钉子而未悬挂。
"""})

> 15/15
{
  "亲访要求": {
    "其他关注点": [
      "核实贷款申请用途与实际经营需求匹配度",
      "关注申请人婚姻状况及家庭资产状况对贷款偿还能力的影响",
      "关注现场人员对贷款访问的反应，排查多头贷款风险",
      "确认公司招牌等可见标识设置情况"
    ],
    "地址验证": [
      "核实亲访地址与营业执照地址是否一致，不一致需提供最新租赁合同及交租凭证",
      "确认经营场所实际存在且环境符合经营需求"
    ],
    "经营稳定性": [
      "确认主营业务占比，评估负责人对业务细节的了解程度"
    ],
    "经营证明": [
      "查看公司招牌、办公环境及员工在岗情况，确认实际经营状态",
      "核查近期经营单据、项目合同，确保与主营业务相关且真实有效"
    ],
    "财务与权属证明": [
      "核查租赁合同、租金支付凭证等财务证明的时效性及一致性",
      "核查银行流水大额资金流动的合理性"
    ]
  }
}
CPU times: user 27.3 ms, sys: 0 ns, total: 27.3 ms
Wall time: 7.22 s


In [19]:
%%time
run({"ID": 16,
"RECORD": """
xxx建议通过，具体评估情况如下：
1. 申请人处于真实经营状态，日常经营主要以丈夫为主导。
2. 申请人的公司与yyy公司存在合作关系，主营业务为硅胶成人用品。公司从yyy公司进货，无需囤积货物及储备原材料，每月固定向yyy公司支出3万元（包含房租及原材料费用）。
3. 申请人拟申请35万元贷款，用于扩大经营规模。申请资料中附有申请人有意租赁的场地照片。
4. 公司注册地址与实际经营地址不一致。
5. 申请人非A城市户籍，拥有2台小车，户口所在地为xx，在B城市拥有房产。
据申请人丈夫介绍，张女士对公司具体运作模式了解不够深入，建议进一步核查其配偶相关信息。
"""})

> 16/16
{
  "亲访要求": {
    "其他关注点": [
      "核实贷款申请用途与实际经营需求匹配度",
      "关注申请人婚姻状况、家庭资产状况及户籍情况对贷款偿还能力的影响",
      "关注现场人员对贷款访问的反应，排查多头贷款风险",
      "确认公司招牌等可见标识设置情况"
    ],
    "地址验证": [
      "核实亲访地址与营业执照地址是否一致，不一致需提供最新租赁合同及交租凭证",
      "确认经营场所实际存在且环境符合经营需求"
    ],
    "经营稳定性": [
      "确认主营业务占比，评估负责人及主要经营者对业务细节的了解程度"
    ],
    "经营证明": [
      "查看公司招牌、办公环境及员工在岗情况，确认实际经营状态",
      "核查近期经营单据、项目合同，确保与主营业务相关且真实有效"
    ],
    "财务与权属证明": [
      "核查租赁合同、租金支付凭证等财务证明的时效性及一致性",
      "核查银行流水大额资金流动的合理性"
    ]
  }
}
CPU times: user 17.3 ms, sys: 189 μs, total: 17.5 ms
Wall time: 7.23 s


### 2.5.6 作假案例

**这里传入作假案例，该案例会展示更多的实操需要关注的细节，如场地破旧、无员工等等。**


In [20]:
%%time
run({"ID": 17,
"RECORD": """
xxx建议拒绝，相关申请已提交，具体情况如下：
该人员为本科学历，已婚且有子女。
1. 其从事政府分包的各类工程业务，公司主营业务为数据服务，例如政府探测数据统计、人口普查等项目。
2. 未能提供大额业务单据，并坦言公司自身资质不足，业务主要通过其他公司分包获取。
3. 存在房屋抵押情况。
4. 公司注册地址与租赁地址（已过期）虽标注为同一地址的601室，但实际不一致。现场核查时，该人员坚称地址无误。
5. 经营场地破旧昏暗，电梯内无公司标识，未悬挂公司招牌（现场为另一家公司标识），无员工办公，相关业务材料从较为空旷的文件框内某文件包中取出。
6. 拟申请50万元贷款，用于外包人工费用垫付。
"""})

> 17/17
{
  "亲访要求": {
    "其他关注点": [
      "核实贷款申请用途与实际经营需求匹配度，贷款用于外包人工费用垫付需说明合理性",
      "关注申请人婚姻状况、家庭资产状况及户籍情况对贷款偿还能力的影响",
      "关注现场人员对贷款访问的反应，排查多头贷款风险",
      "确认公司招牌、标识等可见标识设置情况，现场无标识或标识错误需整改"
    ],
    "地址验证": [
      "核实亲访地址与营业执照地址是否一致，不一致需提供最新租赁合同及交租凭证，现场地址必须实际存在且环境符合经营需求",
      "确认经营场所电梯、门口等显眼位置是否悬挂公司招牌或标识"
    ],
    "经营稳定性": [
      "确认主营业务占比，评估负责人及主要经营者对业务细节的了解程度，分包业务需说明资质及来源",
      "核查公司是否存在资质不足、依赖分包等经营风险"
    ],
    "经营证明": [
      "查看公司招牌、办公环境及员工在岗情况，确认实际经营状态，现场需有员工办公及业务相关材料",
      "核查近期经营单据、项目合同，确保与主营业务相关且真实有效，大额业务单据必须提供"
    ],
    "财务与权属证明": [
      "核查租赁合同、租金支付凭证等财务证明的时效性及一致性，过期租赁合同需补充最新材料",
      "核查银行流水大额资金流动的合理性，房屋抵押情况需说明用途及影响"
    ]
  }
}
CPU times: user 15.8 ms, sys: 1.47 ms, total: 17.3 ms
Wall time: 12.2 s


### 2.5.7 优秀案例

**这里传入较为正向的优秀案例。**


In [21]:
%%time
run({"ID": 18,
"RECORD": """
xxx建议通过，具体评估情况如下：
该人员为大专学历，已婚且有孩子。
1. 申请人处于真实经营状态，主营业务为安防用品，供应对象涵盖企业及政府部门，能够提供完整的进出货单据。
2. 拟申请30万元贷款，用于新店开设及扩大经营规模，计划新店选址于A城市某中心区。
3. 门店地址与实际经营地址一致，但门店招牌名称与公司注册名称不符。
4. 申请人拥有A城市户口，房产位于xx。
5. 能够提供有效的租赁合同。
"""})

> 18/18
{
  "亲访要求": {
    "其他关注点": [
      "核实贷款申请用途与实际经营需求匹配度",
      "关注申请人婚姻状况、家庭资产状况及户籍情况对贷款偿还能力的影响",
      "关注现场人员对贷款访问的反应，排查多头贷款风险"
    ],
    "地址验证": [
      "核实亲访地址与营业执照地址是否一致，不一致需提供最新租赁合同及交租凭证",
      "确认经营场所显眼位置是否悬挂公司招牌或标识，招牌名称需与公司注册名称一致"
    ],
    "经营稳定性": [
      "确认主营业务占比，评估负责人及主要经营者对业务细节的了解程度"
    ],
    "经营证明": [
      "查看公司招牌、办公环境及员工在岗情况，确认实际经营状态",
      "核查近期经营单据、项目合同，确保与主营业务相关且真实有效"
    ],
    "财务与权属证明": [
      "核查租赁合同、租金支付凭证等财务证明的时效性及一致性",
      "核查银行流水大额资金流动的合理性（如有提及）"
    ]
  }
}
CPU times: user 9.58 ms, sys: 7.6 ms, total: 17.2 ms
Wall time: 7.24 s


### 2.5.8 谨慎案例

**特别的，这里传入的案例，需要更为谨慎地判断，就算是较为资深的业务人员都不一定快速鉴别，我们希望LLM模型能更好掌握。**


In [22]:
%%time
run({"ID": 19,
"RECORD": """
xxx建议拒绝，相关申请已提交，具体情况如下：
1. 申请人早期从事通讯业务，2019年开设现电子公司，原计划生产电动汽车，因疫情影响未能实现，后转型生产专利（刚申请尚未获批）产品。
2. 申请人称所在地为B公司所在地，但表示不认识该公司，且自身无门禁权限，称该注册地址系代注册公司提供，仅用于会客。
3. 股东在xx进行代生产，主要由申请人本人出资，其房产和车辆均已处置，累计投入约100万元。
4. 当前销售渠道依赖经销商、地推和网店，月销量几百个，暂无库存。
5. 拟申请35万元贷款，用于产品生产。
"""})

> 19/19
{
  "亲访要求": {
    "其他关注点": [
      "核实贷款申请用途与实际经营需求匹配度，针对拟申请贷款用于尚未获批专利产品生产的情况，需特别评估风险；关注申请人婚姻状况、家庭资产状况（考虑到房产和车辆已处置）及户籍情况对贷款偿还能力的影响；关注现场人员对贷款访问的反应，排查多头贷款风险；记录申请人对注册地址、业务转型等关键问题的解释和态度"
    ],
    "地址验证": [
      "核实亲访地址与营业执照地址是否一致，不一致需提供最新租赁合同及交租凭证，并确认经营场所显眼位置是否悬挂公司招牌或标识，招牌名称需与公司注册名称一致；针对提供地址为代注册公司地址的情况，需特别记录并评估风险"
    ],
    "经营稳定性": [
      "确认主营业务占比，评估负责人及主要经营者对业务细节的了解程度；关注申请人历史业务转型原因及当前业务稳定性"
    ],
    "经营证明": [
      "查看公司招牌、办公环境及员工在岗情况，确认实际经营状态；核查近期经营单据、项目合同，确保与主营业务相关且真实有效；关注申请人当前主营业务及转型情况，评估其可行性和市场前景"
    ],
    "财务与权属证明": [
      "核查租赁合同、租金支付凭证等财务证明的时效性及一致性；针对申请人房产和车辆已处置的情况，需核查其资金流向及当前财务状况"
    ]
  }
}
CPU times: user 26.1 ms, sys: 1.12 ms, total: 27.3 ms
Wall time: 10 s


In [23]:
%%time
run({"ID": 20,
"RECORD": """
xxx建议谨慎通过，具体评估情况如下：
1. 申请人处于实际经营状态，主营业务为加工工艺品。周边餐饮店反馈，该工厂在此地已运营20年。
2. 申请人称公司无租赁合同留存，目前在xx设有新厂房，且新厂房为独资运营。
3. 申请人位于A城市的厂房目前仅作为展厅使用，楼下及一楼已分别出租给其他企业。
4. 申请人目前仅有1名员工在A城市厂房，其余员工均在xx新厂房工作。无法确认A城市厂房是否确实仅保留展厅功能。
5. 申请人已婚并育有子女，全家均无A城市户口，在老家拥有房产，在A城市拥有小车。
申请人拟申请50万元贷款。建议谨慎通过，原因在于无法确认新厂房是否确实位于xx，且A城市B现有厂房的实际经营情况尚不明确。
"""})

> 20/20
{
  "亲访要求": {
    "其他关注点": [
      "核实贷款申请用途与实际经营需求匹配度，评估相关风险",
      "关注申请人婚姻状况、家庭资产状况及户籍情况对贷款偿还能力的影响",
      "关注现场人员对贷款访问的反应，排查多头贷款风险",
      "记录申请人对注册地址、业务转型、新厂房位置等关键问题的解释和态度"
    ],
    "地址验证": [
      "核实亲访地址与营业执照地址是否一致，不一致需提供最新租赁合同及交租凭证",
      "确认经营场所显眼位置是否悬挂公司招牌或标识，招牌名称需与公司注册名称一致",
      "针对提供地址为代注册公司地址或新厂房地址的情况，需特别记录实际地址并评估风险"
    ],
    "经营稳定性": [
      "确认主营业务占比，评估负责人及主要经营者对业务细节的了解程度",
      "关注申请人历史业务转型原因及当前业务稳定性，特别是涉及厂房功能变更的情况"
    ],
    "经营证明": [
      "查看公司招牌、办公环境及员工在岗情况，确认实际经营状态",
      "核查近期经营单据、项目合同，确保与主营业务相关且真实有效",
      "关注申请人当前主营业务，并评估其市场前景"
    ],
    "财务与权属证明": [
      "核查租赁合同、租金支付凭证等财务证明的时效性及一致性",
      "针对申请人房产和车辆情况，需核查其当前财务状况及资产处置情况"
    ]
  }
}
CPU times: user 10 ms, sys: 7.74 ms, total: 17.8 ms
Wall time: 10.3 s


# 三、结果应用

**据此，我们固然可以追加更多亲访记录，用文本迭代来“打磨”要求。可文字再精确，也替代不了现场。一线业务人员，尤其是新人，在面对复杂场景时，仍会把注意力错配、漏掉关键。  
与其再写一段“更好的文字”，不如让 LLM 直接输出一张“会呼吸”的亲访表单：要素顺序、填写提示、风险预警全部排版到位，业务人员“按图索骥”即可，把有限的精力真正留给眼睛、耳朵和脚底板。**

## 3.1 生成表单

In [26]:


def make_form(_criteria):
    client = OpenAI(
        api_key=API_KEY, 
        base_url="https://aistudio.baidu.com/llm/lmapi/v3",
    )

    _system = """
    你是一个信贷风险专家，现在将为不同的中小企业审批下发贷款申请。
    业务人员会去到对应企业的实际生产经营场地亲身访问，以下简称亲访。
    """
    response = client.chat.completions.create(
        model="ernie-4.5-turbo-128k-preview",
        messages=[
            {"role": "system", "content": _system},
            {"role": "user", "content": f"请根据亲访要求:{_criteria}, 生成一个亲访表单，使用标准Markdown格式。"},
        ],

        stream=False,
        extra_body={
            "penalty_score": 1
        },
        max_completion_tokens=12000,
        temperature=0.9, # update
        top_p=0.8,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content

%time result = make_form(criteriaR[20])


CPU times: user 14.4 ms, sys: 6.92 ms, total: 21.3 ms
Wall time: 22.1 s


## 3.2 打印表单

In [27]:

# pip install rich
from rich.console import Console
from rich.markdown import Markdown

console = Console()
console.print(Markdown(result))


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                              中小企业贷款亲访表单                                               ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛


                                                     亲访信息                                                      

亲访日期： ____________ 亲访人员： ____________ 企业名称： ____________ 申请人姓名： ____________ 联系电话：       
____________                                                                                                       


                                                     地址验证                                                      

 • [ ] 营业执照地址与亲访地址一致性：                                                                              
    • □ 一致                                                                                                       
    • □ 不一致，需提供最新租赁合同及交租凭证（请附文件）                                                           
 • [ ] 经营场所显眼位置悬挂公司招牌或标识：                                                                        
    • □ 招牌名称与公司注册名称一致                                                                                 
    • □ 招牌名称与公司注册名称不一致，请记录原因：____________                                                     
 • [ ] 特殊地址情况记录：                                                                                          
    • □ 代注册公司地址                                                                                             
    • □ 新厂房地址                                                                                                 
    • □ 其他（请说明）：____________                                                                               
    • 实际地址：____________                                                                                       
    • 风险评估：____________                                                                                       


                                                     经营证明                                                      

 • [ ] 实际经营状态确认：                                                                                          
    • □ 查看公司招牌、办公环境及员工在岗情况                                                                       
    • □ 经营状态描述：____________                                                                                 
 • [ ] 近期经营单据、项目合同核查：                                                                                
    • □ 与主营业务相关且真实有效                                                                                   
    • □ 经营单据/合同详情：____________                                                                            
 • [ ] 主营业务及市场前景评估：                                                                                    
    • 当前主营业务：____________                                                                                   
    • 市场前景评估：____________                                                                                   


                                                    经营稳定性                                                     

 • [ ] 主营业务占比及业务细节了解程度：                                                                            
    • 主营业务占比：____________                                                                                   
    • 负责人及主要经营者对业务细节的了解程度：                                                                     
       • □ 非常了解                                                                                                
       • □ 基本了解                                                                                                
       • □ 不太了解                                                                                                
       • □ 其他（请说明）：____________                                                                            
 • [ ] 历史业务转型原因及当前业务稳定性：               

# 写在最后

关于智能金融方向，有兴趣的朋友们可以查阅[智能金融之决策科学篇](https://aistudio.baidu.com/projectdetail/8623759)，希望可以帮到大家。

更多的，欢迎关注[AI Studio](https://aistudio.baidu.com/personalcenter/thirdview/979775)/[GitHub](https://github.com/IvanaXu)。
